In [ ]:
#下载微信文章到数据库，批量版
#v3.0.1 2020.11.17-17.29 添加新文章

In [1]:
import os
os.getcwd()

'G:\\xampp\\htdocs\\IELTS\\py'

# New try

## get links

In [2]:
#预备 测试url 获取

#如何批量获取微信文章的url?
import requests
from lxml import html

#1.请求数据

###############
#1.手工访问该页面(这样不好，这是临时链接，最好登陆微信公众号，使用永久链接)
# https://weixin.sogou.com/weixin?type=2&s_from=input&query=英语读报&ie=utf8&_sug_=n&_sug_type_= 
# 右侧点击公众号
#2. 然后F12，把html全部从控制台复制出来。
#3. cat命令保存到/home/wangjl/web/urls.html文件中。 
###############

# urls="http://y.biomooc.com/wangjl/urls.html"

def getURLs(url):
    page=requests.Session().get(url)
    #print("==ok  http://y.biomooc.com/wangjl/urls.html")


    #2.解析数据：获取批量URL
    tree=html.fromstring(page.text)

    result=tree.xpath('//@href')#重复出现2次，所以只取奇数即可。
    arrURL=[]
    for i in range( len(result) ):
        if i==0: continue;
        arrURL.append(result[i])
    print(len(arrURL))
    print(arrURL[0:5])
    return arrURL
# test
url="http://ielts.dawneve.cc/en/contents.html"
arrURL=getURLs(url)
print("="*20, "预期会有n个url，请手工测试一个url能否打开。")

624
['https://mp.weixin.qq.com/s?__biz=MzI0NDcxNzc5Mg==&mid=2247486638&idx=1&sn=b7189e05a0332ba5a23b5492c15912ac&chksm=e958cd21de2f4437e7b072a55eacb285b6ae430c78e892f2ea96b83bdb9fdb64b30c7368358f#rd', 'https://mp.weixin.qq.com/s?__biz=MzI0NDcxNzc5Mg==&mid=2247486633&idx=1&sn=59c2b9d629a6b469ab6a727017e2b7b5&chksm=e958cd26de2f4430cf9f351232ee8db890904a15e7b59419bf95a765770b516fc5e45111f97f#rd', 'https://mp.weixin.qq.com/s?__biz=MzI0NDcxNzc5Mg==&mid=2247486628&idx=1&sn=9eec93e1a1b4a8bcb048cfa5135b650e&chksm=e958cd2bde2f443df1d255a34e3fc23aecf369a6dba4d2d2c602f6e95de217eb4a81d776bd0a#rd', 'https://mp.weixin.qq.com/s?__biz=MzI0NDcxNzc5Mg==&mid=2247486622&idx=1&sn=269de1dcc1f7b2e9f4d180e132cc5fd7&chksm=e958cd11de2f440721cef7c3159a291e8d331b483d0027bfe453a6b547349fe1f49b4fc93962#rd', 'https://mp.weixin.qq.com/s?__biz=MzI0NDcxNzc5Mg==&mid=2247486617&idx=1&sn=1051da63f62b891b2b68955356b9e53c&chksm=e958cd16de2f44003f5cba20e84c7ad4edf9c2bfba739158c0d314516e78e619bdf13f57caa0#rd']
===============

## get full text

In [84]:
# 函数库
import re, requests, sys, pymysql

# v0.2 修改日期获取方式
# v0.3 修正条件:过滤掉日期中的html标签
# v0.4 过滤掉日期中的空格实体

def getInfoByURL2(msg_url, fromSys=False):
    #####################################
    #1.爬虫获取信息
    #####################################
    # 发出请求
    response=requests.get(msg_url)
    assert response.status_code==200,'Error: requests.get'
    #print(respose.text)
    #解析数据
    page3=response.text.encode('utf-8').decode('utf-8')
    

    #1.msg_title
    title=re.findall(r"msg_title = '(.*?)'",page3,re.S)[0]
    #print('title=',title)

    #2.msg_desc
    msg_desc=re.findall(r'msg_desc = "(.*?)"',page3,re.S)[0]
    #print('msg_desc=',msg_desc)

    #3.js_content
    rs=re.findall(r'id="js_content" style="visibility: hidden;"\>(.*?)\<\/div\>',page3,re.S)[0]
    #print('rs=', rs)
    
    # data 
    msg_date=re.findall(r',s="(.*?)"',page3,re.S)[0]

    
    
    #4.获取日期
    
    ########### 方法1，从内容中获取日期
    #aid1 由第一行获取数组：日期字符串，单词数
    def getDateStrSize(strs):
        d1=strs   #rs[1:300];d1
        d2=re.split("words",d1)[0]
        d3=re.split(">",d2)[-1]
        d4=re.split("\|", d3)

        return [d4[0].strip(), int(d4[1].strip())] ##'Aug 30, 2018'日期; 859 单词数
    #aid2 由日期字符串'Aug 30, 2018'获得日期数字字符串 '20180830'
    def getDateFromStr(s):
        a = s; #d4[0].strip()
        month=a[0:3]; month
        year=a[-4:];year
        day=re.split("[\s,]",a)[1];day
        #
        m_dict={"Jan":"01", "Feb":"02", "Mar":"03", "Apr":"04", "May":"05", "Jun":"06", 
                "Jul":"07", "Aug":"08", "Sep":"09", "Oct":"10", "Nov":"11", "Dec":"12" }
        #
        def to2(day):
            if len(day)==1:
                day="0"+day
            return day;

        return year+m_dict[month]+to2(day)
    #
    
    # 根据情况判断使用哪种方法
    raw=re.sub('<[^>]+>','', rs[1:300])
    raw=re.sub('&nbsp;', '', raw)
    if fromSys==False and (len(re.split("\|",raw))>1) and (len(re.split("words",raw))>1)  :
        #print(raw)
        d_n=getDateStrSize(raw);
        #print(d_n)  ##['Aug 30, 2018', 859]
        dateN=getDateFromStr(d_n[0]) #'20180830'
        #5.获取字数
        size=d_n[1]
        #print(dateN, size)
    else:
        ########### 方法2，从系统中获取日期。没有字数信息。
        dateN=re.sub('-','',msg_date)
        size=0
    
    
    #写入文件
    with open('backup/'+dateN+'.html','w', encoding="utf8") as f:
        f.write('<meta http-equiv="Content-Type" content="text/html; charset=utf-8">')
        f.write(title+"<br>")
        f.write(msg_desc)
        f.write(rs)
    print("Title: ", title)
    print("write in to file: http://y.biomooc.com/wangjl/en"+dateN+".html")
    
    return [url,dateN, size,title,msg_desc,rs]

In [4]:
url="https://mp.weixin.qq.com/s?__biz=MzI0NDcxNzc5Mg==&mid=2247486638&idx=1&sn=b7189e05a0332ba5a23b5492c15912ac&chksm=e958cd21de2f4437e7b072a55eacb285b6ae430c78e892f2ea96b83bdb9fdb64b30c7368358f#rd"
result=getInfoByURL2(url)

Title:  economist | Child-safe internet: Kids aren&#39;t alright
write in to file: http://y.biomooc.com/wangjl/en20190626.html


## into DB

In [89]:
# v0.2 desc 最长150字符
def insertInfo2DB2(info,source="英语读报"):
    # 打开数据库连接
    db = pymysql.connect(host='y.biomooc.com',port=7070,user='root', password='123456', \
                         database='wang', charset='utf8')

    # 使用cursor()方法获取操作游标 
    cursor = db.cursor()

    #如果没有，则保存到数据库中
    def DoInsertDB(info,source=source):
        # SQL 插入语句
        sql = "INSERT INTO msg_English(msg_url,msg_origin,msg_date, msg_size, msg_title, msg_desc,msg_content) \
               VALUES ('%s', '%s', '%s', '%d', '%s', '%s','%s' )" % \
              (info[0],source, info[1],info[2],
                   pymysql.escape_string(info[3]),
                   pymysql.escape_string(info[4])[1:150],
                   pymysql.escape_string(info[5]))
                #[url,dateN, size,title,msg_desc,rs]
               #(msg_url, '英语读报', dateN, size, title, msg_desc, pymysql.escape_string(rs))
        try:
            cursor.execute(sql) # 执行sql语句
            db.commit() # 提交到数据库执行
            print("commit successfully")
        except:
            rsponse=db.rollback() # 发生错误时回滚
            print("rollback")
            print( sys.exc_info() )

    #如果日期已经存在，则不进行处理


    # SQL 查询语句
    dateN2=info[1]
    #dateN2="20170102"
    sql = 'select msg_date from msg_English where msg_date like "'+dateN2+'";'

    # 执行sql语句
    cursor.execute(sql)
    values = cursor.fetchall()
    if len(values)==0:
        print("Not exist the date")
        DoInsertDB(info,source)
    else:
        print("The date "+dateN2+" has already existed.")

    # 关闭数据库连接
    cursor.close()
    db.close()

## one function

In [6]:
# 批量URL获取文章，并插入数据库
import time,random

for i in range(len(arrURL)): #随机时间段后[2s,12s]执行
#for i in range(4,6):    
    msg_url=arrURL[i]
    print(msg_url)
    #2.get Info  return [url,dateN, size,title,msg_desc,rs]
    info= getInfoByURL2(msg_url)

    #3.插入数据库
    insertInfo2DB2(info)
    
    #等待随机时间
    pause=2+2*random.random()
    print(str(i)+' '+str(pause))
    time.sleep(pause)
    print('='*20)
#++

https://mp.weixin.qq.com/s?__biz=MzI0NDcxNzc5Mg==&mid=2247486638&idx=1&sn=b7189e05a0332ba5a23b5492c15912ac&chksm=e958cd21de2f4437e7b072a55eacb285b6ae430c78e892f2ea96b83bdb9fdb64b30c7368358f#rd
Title:  economist | Child-safe internet: Kids aren&#39;t alright
write in to file: http://y.biomooc.com/wangjl/en20190626.html
Not exist the date
commit successfully
0 2.3491573631290956
https://mp.weixin.qq.com/s?__biz=MzI0NDcxNzc5Mg==&mid=2247486633&idx=1&sn=59c2b9d629a6b469ab6a727017e2b7b5&chksm=e958cd26de2f4430cf9f351232ee8db890904a15e7b59419bf95a765770b516fc5e45111f97f#rd
Title:  science|What big ideas will shape U.S. science over the next ...
write in to file: http://y.biomooc.com/wangjl/en20190625.html
Not exist the date
commit successfully
1 2.0266596352955597
https://mp.weixin.qq.com/s?__biz=MzI0NDcxNzc5Mg==&mid=2247486628&idx=1&sn=9eec93e1a1b4a8bcb048cfa5135b650e&chksm=e958cd2bde2f443df1d255a34e3fc23aecf369a6dba4d2d2c602f6e95de217eb4a81d776bd0a#rd
Title:  nytimes|U.S. blacklists 5 Chine

IndexError: list index out of range

In [66]:
run(19,20)

https://mp.weixin.qq.com/s?__biz=MzI0NDcxNzc5Mg==&mid=2247486545&idx=2&sn=7a5b7624026694efadfa91bcffbcdfe4&chksm=e958cddede2f44c80a73bfcaa995c31d63e33e150b0515645cdfe4f880fc898e526517366b79#rd
Title:  如何通过英语读报积累写作句型？(5)【BBC:芯片漏洞】
write in to file: http://y.biomooc.com/wangjl/en20190608.html
Not exist the date
commit successfully
19 4.9926601839085585


In [13]:
for i in range(20, len(arrURL)): #随机时间段后[2s,12s]执行
#for i in range(4,6):    
    msg_url=arrURL[i]
    print(msg_url)
    #2.get Info  return [url,dateN, size,title,msg_desc,rs]
    info= getInfoByURL2(msg_url)

    #3.插入数据库
    insertInfo2DB2(info)
    
    #等待随机时间
    pause=3+2*random.random()
    print(str(i)+' '+str(pause))
    time.sleep(pause)
    print('='*20)

https://mp.weixin.qq.com/s?__biz=MzI0NDcxNzc5Mg==&mid=2247486543&idx=1&sn=5fbea3b746381894d4b87bd0e4f88c5e&chksm=e958cdc0de2f44d6ca790c2bfc195b7a73fe92987c672414c9ca37b5f43632d4a89d4dce23d5#rd
Title:  science | The calm before the storms
write in to file: http://y.biomooc.com/wangjl/en20190607.html
Not exist the date
commit successfully
20 3.200852597321046
https://mp.weixin.qq.com/s?__biz=MzI0NDcxNzc5Mg==&mid=2247486541&idx=1&sn=e073441216cab1ca41a9f8004d85cd53&chksm=e958cdc2de2f44d4699aa169cfab75941daf08341f67959e9ada15ff9d53f2a6dcf695ee4e50#rd
Title:  guardian | How Monsanto manipulates journalists and academics
write in to file: http://y.biomooc.com/wangjl/en20190606.html
Not exist the date
commit successfully
21 3.263427958004975
https://mp.weixin.qq.com/s?__biz=MzI0NDcxNzc5Mg==&mid=2247486539&idx=1&sn=725b7ea9a6b2342641160276a4b988ed&chksm=e958cdc4de2f44d2b7357126bc3d360d55b9b25c7a17704ca0995d6bd9581fbc7d5071a1ad5e#rd
Title:  Guardian | If Netflix decided to show ads, revenues re

Title:  Times|Natural fruit juices match fizzy drinks for sugar dangers
write in to file: http://y.biomooc.com/wangjl/en20190518.html
Not exist the date
commit successfully
40 3.60662552002397
https://mp.weixin.qq.com/s?__biz=MzI0NDcxNzc5Mg==&mid=2247486427&idx=1&sn=a01608bd47dd5836eaea00c29781395a&chksm=e958ca54de2f434221d27e7c138c2c79b96bbb27f9c14835daf55d61eaf199962022634a862c#rd
Title:  forbes | Beyond The Smog: What Does Our Future Energy System ...
write in to file: http://y.biomooc.com/wangjl/en20190517.html
Not exist the date
commit successfully
41 3.653635230564735
https://mp.weixin.qq.com/s?__biz=MzI0NDcxNzc5Mg==&mid=2247486426&idx=1&sn=070fa602ca15b30f77dedc68c2962e3f&chksm=e958ca55de2f434347eb960461af753fe331a4b2a29d14eab40799529c892fd997de09bfd787#rd
Title:  BBC | Intel Zombieload bug fix to slow data centre computers
write in to file: http://y.biomooc.com/wangjl/en20190516.html
Not exist the date
commit successfully
42 3.591290396097615
https://mp.weixin.qq.com/s?__biz=Mz

IndexError: list index out of range

In [68]:
run(43,44)

https://mp.weixin.qq.com/s?__biz=MzI0NDcxNzc5Mg==&mid=2247486426&idx=2&sn=8d782b5bf82a221d825f6b06d6a1aad2&chksm=e958ca55de2f43439216d629e33a6b8843f503b1eeaae24b77c9989077049b29afeb1c1531e4#rd
Title:  如何通过英语读报积累写作句型？(4)【economist:艺术史】
write in to file: http://y.biomooc.com/wangjl/en20190516.html
Not exist the date
commit successfully
43 3.7244437972436533


In [10]:
msg_url

'https://mp.weixin.qq.com/s?__biz=MzI0NDcxNzc5Mg==&mid=2247486545&idx=2&sn=7a5b7624026694efadfa91bcffbcdfe4&chksm=e958cddede2f44c80a73bfcaa995c31d63e33e150b0515645cdfe4f880fc898e526517366b79#rd'

In [ ]:
# 不能出现中文
# 'https://mp.weixin.qq.com/s?__biz=MzI0NDcxNzc5Mg==&mid=2247486545&idx=2&sn=7a5b7624026694efadfa91bcffbcdfe4&chksm=e958cddede2f44c80a73bfcaa995c31d63e33e150b0515645cdfe4f880fc898e526517366b79#rd'
# https://mp.weixin.qq.com/s?__biz=MzI0NDcxNzc5Mg==&mid=2247486426&idx=2&sn=8d782b5bf82a221d825f6b06d6a1aad2&chksm=e958ca55de2f43439216d629e33a6b8843f503b1eeaae24b77c9989077049b29afeb1c1531e4#rd
# https://mp.weixin.qq.com/s?__biz=MzI0NDcxNzc5Mg==&mid=2247486411&idx=2&sn=a6e4cb50296856aac414bb4fccfae418&chksm=e958ca44de2f4352ed904e1dca8c4ecf6bac2a9e0e793fddf265dab7e08b0a34dd95ea1364a6#rd


In [85]:
def run(start, end=-1, fromSys=False):
    if end==-1:
        end=len(arrURL)
    for i in range(start, end): #随机时间段后[2s,12s]执行
    #for i in range(4,6):    
        msg_url=arrURL[i]
        print(msg_url)
        #2.get Info  return [url,dateN, size,title,msg_desc,rs]
        info= getInfoByURL2(msg_url, fromSys)

        #3.插入数据库
        insertInfo2DB2(info)

        #等待随机时间
        pause=3+2*random.random()
        print(str(i)+' '+str(pause))
        time.sleep(pause)
        print('='*20)

In [60]:
run(44)

https://mp.weixin.qq.com/s?__biz=MzI0NDcxNzc5Mg==&mid=2247486411&idx=1&sn=cff376c390720d54831d659ce1463b99&chksm=e958ca44de2f435286c0c0ac4749061f5f6091e6ab69a747f6c61a1cb4347ecc32d4fca7da91#rd
Title:  economist | Art history: A brave new world
write in to file: http://y.biomooc.com/wangjl/en20190515.html
The date 20190515 has already existed.
44 4.354732656077292
https://mp.weixin.qq.com/s?__biz=MzI0NDcxNzc5Mg==&mid=2247486411&idx=2&sn=a6e4cb50296856aac414bb4fccfae418&chksm=e958ca44de2f4352ed904e1dca8c4ecf6bac2a9e0e793fddf265dab7e08b0a34dd95ea1364a6#rd


KeyboardInterrupt: 

In [70]:
run(45,46)

https://mp.weixin.qq.com/s?__biz=MzI0NDcxNzc5Mg==&mid=2247486411&idx=2&sn=a6e4cb50296856aac414bb4fccfae418&chksm=e958ca44de2f4352ed904e1dca8c4ecf6bac2a9e0e793fddf265dab7e08b0a34dd95ea1364a6#rd
Title:  如何通过英语读报积累写作句型？(3) 【NPR: 放大的疼痛】
write in to file: http://y.biomooc.com/wangjl/en20190515.html
Not exist the date
commit successfully
45 4.650376318231643


In [15]:
run(46)

https://mp.weixin.qq.com/s?__biz=MzI0NDcxNzc5Mg==&mid=2247486409&idx=1&sn=c0d7b61dae264291b3307930d4370b1f&chksm=e958ca46de2f4350bee73f8e6bfbf6fb0c5c51778c290e6987812f6e69d6fc707da2c2de3f9e#rd
Title:  NPR | Invisibilia: For Some Teens With Debilitating Pain, The...
write in to file: http://y.biomooc.com/wangjl/en20190514.html
Not exist the date
commit successfully
46 3.0283179541066545
https://mp.weixin.qq.com/s?__biz=MzI0NDcxNzc5Mg==&mid=2247486404&idx=1&sn=702ae5734ec4fa12e727f3dfa5114fea&chksm=e958ca4bde2f435d0379aea992eef5d26c120930e435857d29f6c518dc2f519e8bbf24abb6da#rd
Title:  economist|Two books explore the evolutionary origins of morality
write in to file: http://y.biomooc.com/wangjl/en20190513.html
Not exist the date
commit successfully
47 4.376090486661854
https://mp.weixin.qq.com/s?__biz=MzI0NDcxNzc5Mg==&mid=2247486404&idx=2&sn=100aae2456dce4b45fe807da65d419eb&chksm=e958ca4bde2f435da3b53155be1bd0eee5f774e4f5bdf88fdd3929ca6d51c28af5ab9cc31354#rd


IndexError: list index out of range

In [71]:
run(48,49)

https://mp.weixin.qq.com/s?__biz=MzI0NDcxNzc5Mg==&mid=2247486404&idx=2&sn=100aae2456dce4b45fe807da65d419eb&chksm=e958ca4bde2f435da3b53155be1bd0eee5f774e4f5bdf88fdd3929ca6d51c28af5ab9cc31354#rd
Title:  如何通过英语读报积累写作句型？(2) 【Atlantic: 权游配乐】
write in to file: http://y.biomooc.com/wangjl/en20190513.html
Not exist the date
commit successfully
48 4.175544319337756


In [16]:
run(49)

https://mp.weixin.qq.com/s?__biz=MzI0NDcxNzc5Mg==&mid=2247486402&idx=1&sn=d78a44821e8a3a45f0be219bb52405aa&chksm=e958ca4dde2f435b93eb556f74d0850816e0ea6c2d166a9001fcc90225b26e9f9cffa30570f9#rd
Title:  economist | The “bad grandpas” and the last great British heist
write in to file: http://y.biomooc.com/wangjl/en20190512.html
Not exist the date
commit successfully
49 3.3163977853368665
https://mp.weixin.qq.com/s?__biz=MzI0NDcxNzc5Mg==&mid=2247486402&idx=2&sn=79b61e7be76fbf4d7e9541ab47c8d670&chksm=e958ca4dde2f435b05d80f1fb4050a24b15820f397bea83bdb2d4b6f3fe4202cbf672cf9598b#rd


IndexError: list index out of range

In [72]:
run(50,51)

https://mp.weixin.qq.com/s?__biz=MzI0NDcxNzc5Mg==&mid=2247486402&idx=2&sn=79b61e7be76fbf4d7e9541ab47c8d670&chksm=e958ca4dde2f435b05d80f1fb4050a24b15820f397bea83bdb2d4b6f3fe4202cbf672cf9598b#rd
Title:  如何通过英语读报积累写作句型？(1)
write in to file: http://y.biomooc.com/wangjl/en20190512.html
Not exist the date
commit successfully
50 4.0464765017109645


In [17]:
run(51)

https://mp.weixin.qq.com/s?__biz=MzI0NDcxNzc5Mg==&mid=2247486399&idx=1&sn=30db8cde439902b4f326609bad5aaabc&chksm=e958ca30de2f43261d61d241ea90bccd7b937de7a43a8f17a9fe14e5c47503e7c0c0af6945bc#rd
Title:  economist | Mark Zuckerberg wants to build WeChat for the West
write in to file: http://y.biomooc.com/wangjl/en20190511.html
Not exist the date
commit successfully
51 4.8856298233395705
https://mp.weixin.qq.com/s?__biz=MzI0NDcxNzc5Mg==&mid=2247486378&idx=1&sn=b31d25c7206f265bf6b81c1c3905b49c&chksm=e958ca25de2f43330977c5119b1a9029df4fb8999493e3b337e250bd6f7117cf45d9386fa634#rd
Title:  npr | When &#39;1-In-100-Year&#39; Floods Happen Often, What Should ...
write in to file: http://y.biomooc.com/wangjl/en20190510.html
Not exist the date
commit successfully
52 4.560555999853147
https://mp.weixin.qq.com/s?__biz=MzI0NDcxNzc5Mg==&mid=2247486373&idx=1&sn=35485c42cd063a02f20d064af4636c2d&chksm=e958ca2ade2f433cfb40058bdc5882ba9f8f7131743be911eda2fc5176e7f313936c614ff752#rd
Title:  economist|Riding 

IndexError: list index out of range

In [ ]:
# 69违规

In [18]:
run(70)

https://mp.weixin.qq.com/s?__biz=MzI0NDcxNzc5Mg==&mid=2247486271&idx=1&sn=e495ccfea2eaa8b71a7eec4878246426&chksm=e958cab0de2f43a6df53dd178d65af9a6369514c7394474ecd696c384547a9fab94849df27e3#rd
Title:  Guardian | Why nightingales are snubbing Berkeley Square for ...
write in to file: http://y.biomooc.com/wangjl/en20190420.html
Not exist the date
commit successfully
70 3.60645749523663
https://mp.weixin.qq.com/s?__biz=MzI0NDcxNzc5Mg==&mid=2247486266&idx=1&sn=a1eea93f5d44fd5fec191f836c12bb25&chksm=e958cab5de2f43a3cefd3daef3449cee2f318ae51651e06ff318748e234093cb75b7854b0761#rd
Title:  BBC | Amazon plans to shut online store in China
write in to file: http://y.biomooc.com/wangjl/en20190419.html
Not exist the date
commit successfully
71 3.3470212547842833
https://mp.weixin.qq.com/s?__biz=MzI0NDcxNzc5Mg==&mid=2247486261&idx=1&sn=5ee0373c7160368f70bde4bbe2a0c58a&chksm=e958cabade2f43acbba594a44feca562d73a929dde2923c03cffc52f9cea3a64c73cd43a0a54#rd
Title:  indianEX|Visualchina watermarks black h

Title:  npr | Are We Ready For An Implant That Can Change Our Moods?
write in to file: http://y.biomooc.com/wangjl/en20190331.html
Not exist the date
commit successfully
90 3.0688001701297303
https://mp.weixin.qq.com/s?__biz=MzI0NDcxNzc5Mg==&mid=2247486160&idx=1&sn=741082f2d257d89df8748083ed504f42&chksm=e958cb5fde2f4249e13f03f47da9eb528fbfbf2586486c925f7bb7cd577c21aef07d02416002#rd
Title:  time|Ride-Share Service Lyft Jumps 21% On First Day of Stock ...
write in to file: http://y.biomooc.com/wangjl/en20190330.html
Not exist the date
commit successfully
91 3.977882453510698
https://mp.weixin.qq.com/s?__biz=MzI0NDcxNzc5Mg==&mid=2247486155&idx=1&sn=a3d46397f423b0c8fb4aaaa171c722da&chksm=e958cb44de2f42521a1797d34fab1f9b58bd14e920ecc5673936512125b8e464d14fea6acb25#rd
Title:  nature | Scientists rise up against statistical significance
write in to file: http://y.biomooc.com/wangjl/en20190329.html
Not exist the date
commit successfully
92 4.110392560808074
https://mp.weixin.qq.com/s?__biz=MzI

https://mp.weixin.qq.com/s?__biz=MzI0NDcxNzc5Mg==&mid=2247486046&idx=1&sn=1899c3b8fc0c82fcf3ce9bee561e0433&chksm=e958cbd1de2f42c767a27afca58b38a276111b14fb47643f551c82a76bfdd2058e2583468892#rd
Title:  science|There’s an arms race going on in this bird nest
write in to file: http://y.biomooc.com/wangjl/en20190310.html
Not exist the date
commit successfully
111 4.987774256050249
https://mp.weixin.qq.com/s?__biz=MzI0NDcxNzc5Mg==&mid=2247486041&idx=1&sn=1eafe01d18ab77fb289ad657c8661ae6&chksm=e958cbd6de2f42c0615216b1b21d96d384f883bb6b99b7efbb15ecf2b9b492e2d8212278ac9b#rd
Title:  cnn|Singapore announces opening date for long-awaited xx Airport
write in to file: http://y.biomooc.com/wangjl/en20190309.html
Not exist the date
commit successfully
112 4.890589512873068
https://mp.weixin.qq.com/s?__biz=MzI0NDcxNzc5Mg==&mid=2247486040&idx=1&sn=8e4bca0c7408c0d11695d2eb633b80a5&chksm=e958cbd7de2f42c1607209cb4603f88b2cb3676f0a32ec56eea5871c6fe1a33815e693c9dcaf#rd
Title:  fortune|Google Celebrates Inte

Title:  hbr | The Price of “Success”
write in to file: http://y.biomooc.com/wangjl/en20190218.html
Not exist the date
commit successfully
131 4.063472452597832
https://mp.weixin.qq.com/s?__biz=MzI0NDcxNzc5Mg==&mid=2247485939&idx=1&sn=a865c1b5e0cc7b907cbee60d41a6786f&chksm=e958c87cde2f416abbcdc1ebccc4da7a34d0a3b70699b67b5d3d2576e7284bfff4e7345655a0#rd
Title:  Verge|China’s blockbuster The Wandering Earth is rich, ...
write in to file: http://y.biomooc.com/wangjl/en20190217.html
Not exist the date
commit successfully
132 4.134282197694723
https://mp.weixin.qq.com/s?__biz=MzI0NDcxNzc5Mg==&mid=2247485932&idx=1&sn=9c17f0ddb4ab5c5be1230c5bd3bdbee5&chksm=e958c863de2f41751fb9363d6df1d34f46ea7011a7fc8909420bd8741781e230f2615300cf7e#rd
Title:  guardian|The EU looks like the Soviet Union in 1991 – on the ...
write in to file: http://y.biomooc.com/wangjl/en20190216.html
Not exist the date
commit successfully
133 3.678928566547931
https://mp.weixin.qq.com/s?__biz=MzI0NDcxNzc5Mg==&mid=2247485923&idx

Title:  wp|The real lessons from the fall of Sears and GE
write in to file: http://y.biomooc.com/wangjl/en20190120.html
Not exist the date
commit successfully
152 3.319300738462979
https://mp.weixin.qq.com/s?__biz=MzI0NDcxNzc5Mg==&mid=2247485845&idx=1&sn=be197e0dc4a819933f68467922cccdae&chksm=e958c81ade2f410c06b804f886e90608743cc13b5a69a067e2c0b94f399733c5fb111a8e3d48#rd
Title:  economist | Permission to speak
write in to file: http://y.biomooc.com/wangjl/en20190119.html
Not exist the date
commit successfully
153 3.2443011099313175
https://mp.weixin.qq.com/s?__biz=MzI0NDcxNzc5Mg==&mid=2247485840&idx=1&sn=e1cc72bae8c555299e8fe871912ecfae&chksm=e958c81fde2f4109856ae294b5e1a192d587ebfa245a0b0d0fadc46aee26616d54d97c405710#rd
Title:  wp|Amazon reportedly developing a subscription video game ...
write in to file: http://y.biomooc.com/wangjl/en20190118.html
Not exist the date
commit successfully
154 3.556480290082008
https://mp.weixin.qq.com/s?__biz=MzI0NDcxNzc5Mg==&mid=2247485831&idx=1&sn=72

Title:  nybooks|In the Valley of Fear
write in to file: http://y.biomooc.com/wangjl/en20181229.html
Not exist the date
commit successfully
173 3.5931479411733456
https://mp.weixin.qq.com/s?__biz=MzI0NDcxNzc5Mg==&mid=2247485708&idx=1&sn=cad11618c69b9ae28228f3089b43112c&chksm=e958c883de2f41956b81d02e7ab41d26378ae7122049f60989bd451b8cc55578b1151f3a2a43#rd
Title:  FT | Microsoft and BMW invest in UK chipmaker Graphcore
write in to file: http://y.biomooc.com/wangjl/en20181228.html
Not exist the date
commit successfully
174 4.590470096082731
https://mp.weixin.qq.com/s?__biz=MzI0NDcxNzc5Mg==&mid=2247485707&idx=1&sn=66d8a8ff176797f1489d9490707b907c&chksm=e958c884de2f4192f9fe7296a652840eac019bb0e1623be6d4457886469f5a98eff4e7553966#rd
Title:  atlantic|China’s Hottest Bachelors Are Animated Characters
write in to file: http://y.biomooc.com/wangjl/en20181227.html
Not exist the date
commit successfully
175 4.617350770249565
https://mp.weixin.qq.com/s?__biz=MzI0NDcxNzc5Mg==&mid=2247485706&idx=1&sn=5

IndexError: list index out of range

In [73]:
run(183,184)

https://mp.weixin.qq.com/s?__biz=MzI0NDcxNzc5Mg==&mid=2247485650&idx=1&sn=44a7b85213d8799bd78a11a45a566564&chksm=e958c95dde2f404b8dc3753ab8f2b40774b5100aaae9ec01340b97738a230f7ad82722d63050#rd
Title:  economist | Obituary: Stan Lee died on November 12th
write in to file: http://y.biomooc.com/wangjl/en20181219.html
Not exist the date
commit successfully
183 3.0802315964538325


In [20]:
run(184)

https://mp.weixin.qq.com/s?__biz=MzI0NDcxNzc5Mg==&mid=2247485644&idx=1&sn=02fa2674aa1e2a4a615e049e611dbb71&chksm=e958c943de2f405507b48d65b12308a4f499e8ad0c76af5d00d2140367c4155079c9eb3fccb9#rd
Title:  atlantic | The Strange Allure of Pioneer Living
write in to file: http://y.biomooc.com/wangjl/en20181218.html
Not exist the date
commit successfully
184 3.765141410307079
https://mp.weixin.qq.com/s?__biz=MzI0NDcxNzc5Mg==&mid=2247485643&idx=1&sn=747b30b7a8212d9adde263367b2f142c&chksm=e958c944de2f4052da6bfb2f43159f7888fba41521b94d27fc25c8490948d252ee40419b70e4#rd
Title:  economist|The meaning of the words of the year
write in to file: http://y.biomooc.com/wangjl/en20181217.html
Not exist the date
commit successfully
185 3.4649485503265582
https://mp.weixin.qq.com/s?__biz=MzI0NDcxNzc5Mg==&mid=2247485642&idx=1&sn=96f31a7ef1f504c2afb0691b5cfbcca6&chksm=e958c945de2f405330f758dc1540b302bdbdd1ca0573d421ddfdd1f9191549ba1b2106da6a3a#rd
Title:  economist|Roman history:To bury, not to praise
write in

https://mp.weixin.qq.com/s?__biz=MzI0NDcxNzc5Mg==&mid=2247485553&idx=1&sn=ebf9041b2141d1f5c17cb22efda16331&chksm=e958c9fede2f40e803e759cea418969526edebd2e14ccd1540cbb10aede4457e5db8d7f6a5e2#rd
Title:  BB|China Scientist’s Claim on World’s First Gene-Edited Babies
write in to file: http://y.biomooc.com/wangjl/en20181127.html
Not exist the date
commit successfully
205 4.777931442242488
https://mp.weixin.qq.com/s?__biz=MzI0NDcxNzc5Mg==&mid=2247485548&idx=1&sn=bcec18508475e8eb5204a87c4db111ab&chksm=e958c9e3de2f40f50338796c0847b33d9f0246405e7072ea1f917e4630351ce649a82656a3b5#rd
Title:  nature|Quantum computers put blockchain security at risk
write in to file: http://y.biomooc.com/wangjl/en20181126.html
Not exist the date
commit successfully
206 4.75715628065232
https://mp.weixin.qq.com/s?__biz=MzI0NDcxNzc5Mg==&mid=2247485543&idx=1&sn=a8d68a5ed91f5c18e9d14a366c7ff04d&chksm=e958c9e8de2f40fe1719815a23eae8a0627d17be0db04a1fbd4249800da33a1dd6a831378fd2#rd
Title:  Time|Why America’s Founders Trie

Title:  express | Winston Churchill was the greatest leader we have ...
write in to file: http://y.biomooc.com/wangjl/en20181107.html
Not exist the date
commit successfully
225 4.466434237293026
https://mp.weixin.qq.com/s?__biz=MzI0NDcxNzc5Mg==&mid=2247485421&idx=1&sn=64ff2d94accc5cd39bc6c5a09df5afcf&chksm=e958c662de2f4f74c4266307037d9f3b1076b7bf0fba17ee4b51b1e2caf2be1ef488f63979a3#rd
Title:  science|Researchers launch plan to sequence 66,000 species in...
write in to file: http://y.biomooc.com/wangjl/en20181106.html
Not exist the date
commit successfully
226 3.2869384024011086
https://mp.weixin.qq.com/s?__biz=MzI0NDcxNzc5Mg==&mid=2247485416&idx=1&sn=47071b305ba925dada1d9dc7f1e6333a&chksm=e958c667de2f4f7135dfc825e93131c9bdcd2846b1020cca3438914ffc76d60a5d47fcddeb15#rd
Title:  theWeek|Is Apple trying to kill the laptop?
write in to file: http://y.biomooc.com/wangjl/en20181105.html
Not exist the date
commit successfully
227 3.066362308060831
https://mp.weixin.qq.com/s?__biz=MzI0NDcxNzc5Mg

https://mp.weixin.qq.com/s?__biz=MzI0NDcxNzc5Mg==&mid=2247485319&idx=1&sn=66e32fc1c6ed1d413e0e9a377657024b&chksm=e958c608de2f4f1ec5d2e87877ff80b042ade6366a9e13801b90fca373fa597b4ae8ac463ec4#rd
Title:  TheVerge|Microsoft co-founder Paul Allen helped change the world
write in to file: http://y.biomooc.com/wangjl/en20181017.html
Not exist the date
commit successfully
246 3.481033889854632
https://mp.weixin.qq.com/s?__biz=MzI0NDcxNzc5Mg==&mid=2247485305&idx=1&sn=e1cdb738dd9e562846366ee3963efab6&chksm=e958c6f6de2f4fe0d11140c2a1699b41b5583a719a1b83782452257d335c6f6b69f5063330e3#rd
Title:  CNN|Inside &#39;Western Village,&#39; Japan&#39;s eerie abandoned theme park
write in to file: http://y.biomooc.com/wangjl/en20181016.html
Not exist the date
commit successfully
247 3.5650291084178596
https://mp.weixin.qq.com/s?__biz=MzI0NDcxNzc5Mg==&mid=2247485300&idx=1&sn=b27bdc4eba1db0013631b2de85b27ddc&chksm=e958c6fbde2f4fed88bf82dd36523ef5fbb8676d3d2410b93ef631db08c699b453ba4018ae0a#rd
Title:  economis

IndexError: list index out of range

In [74]:
run(251,252)

https://mp.weixin.qq.com/s?__biz=MzI0NDcxNzc5Mg==&mid=2247485288&idx=2&sn=a59c4d6bed8a7750111af7d8b76376c1&chksm=e958c6e7de2f4ff1ec8048ea3b2d1062f8fa53007bafa7f32c192c8497a327b2e5b3ebd2100f#rd
Title:  NCE3 | Lesson 5 The facts 确切数字
write in to file: http://y.biomooc.com/wangjl/en20181013.html
Not exist the date
commit successfully
251 3.9371755526390215


In [21]:
run(252)

https://mp.weixin.qq.com/s?__biz=MzI0NDcxNzc5Mg==&mid=2247485287&idx=1&sn=430eaa119739b559d391f9b9030ba417&chksm=e958c6e8de2f4ffe8241e4a1b0239a28ca4bfa3b0bd74d97edbaa153834b60be821f6ba5710d#rd
Title:  fortune | Black Wednesday? The Day Every Stock Went Down
write in to file: http://y.biomooc.com/wangjl/en20181012.html
Not exist the date
commit successfully
252 3.2700707029562555
https://mp.weixin.qq.com/s?__biz=MzI0NDcxNzc5Mg==&mid=2247485282&idx=1&sn=b1fb1836c647ae7e03a12e1e82be7554&chksm=e958c6edde2f4ffb21fb42a52c5326417b7bc598f8c937a46a41b80b4b3b33ded6b526cae82a#rd
Title:  bloombergQuint|New Evidence of Hacked Supermicro Hardware ...
write in to file: http://y.biomooc.com/wangjl/en20181011.html
Not exist the date
commit successfully
253 4.468464406356656
https://mp.weixin.qq.com/s?__biz=MzI0NDcxNzc5Mg==&mid=2247485282&idx=2&sn=cb69bca62b8172e2eca62b2d8c6929ee&chksm=e958c6edde2f4ffb936a3646b01cac886046fefc778ea352634e1d116735382e4ae8b5e0e6b3#rd


IndexError: list index out of range

In [75]:
run(254, 255)

https://mp.weixin.qq.com/s?__biz=MzI0NDcxNzc5Mg==&mid=2247485282&idx=2&sn=cb69bca62b8172e2eca62b2d8c6929ee&chksm=e958c6edde2f4ffb936a3646b01cac886046fefc778ea352634e1d116735382e4ae8b5e0e6b3#rd
Title:  NCE3|Lesson 6 Smash-and-grab 砸橱窗抢劫
write in to file: http://y.biomooc.com/wangjl/en20181011.html
Not exist the date
commit successfully
254 4.293406456474532


In [23]:
run(255)

https://mp.weixin.qq.com/s?__biz=MzI0NDcxNzc5Mg==&mid=2247485275&idx=1&sn=f830610273a992048213e6162bde70be&chksm=e958c6d4de2f4fc282567a45025f08078cbdbcb8c7b78665c31ec08f841fa12c13a776b2ec3d#rd
Title:  theGlobeAndMail|How GE went from business icon to basket case
write in to file: http://y.biomooc.com/wangjl/en20181010.html
Not exist the date
commit successfully
255 3.305241449693219
https://mp.weixin.qq.com/s?__biz=MzI0NDcxNzc5Mg==&mid=2247485275&idx=2&sn=ff24485b4bde2b3c697cd985a8ea0525&chksm=e958c6d4de2f4fc20067d2aedc1d923879e51fe9eef541fe983c907093c57b7bad9bb6a48515#rd


IndexError: list index out of range

In [76]:
run(256, 257)

https://mp.weixin.qq.com/s?__biz=MzI0NDcxNzc5Mg==&mid=2247485275&idx=2&sn=ff24485b4bde2b3c697cd985a8ea0525&chksm=e958c6d4de2f4fc20067d2aedc1d923879e51fe9eef541fe983c907093c57b7bad9bb6a48515#rd
Title:  NCE3|Lesson 4 The double life of Alfred Bloggs 阿尔弗雷德.布洛格斯的双重生活
write in to file: http://y.biomooc.com/wangjl/en20181010.html
Not exist the date
commit successfully
256 4.094190671797888


In [24]:
run(257)

https://mp.weixin.qq.com/s?__biz=MzI0NDcxNzc5Mg==&mid=2247485273&idx=1&sn=4b0c283aa42abd1038f0922a1ee5fb04&chksm=e958c6d6de2f4fc0571b343cffef0e94f0fde6a48e9ff8010e55a87d91188da047786dbdce5a#rd
Title:  independent|Where’s my baby?How Canada confronted the scandal...
write in to file: http://y.biomooc.com/wangjl/en20181009.html
Not exist the date
commit successfully
257 4.602091031763102
https://mp.weixin.qq.com/s?__biz=MzI0NDcxNzc5Mg==&mid=2247485273&idx=2&sn=7d5c50da130a60612f9827798de921a3&chksm=e958c6d6de2f4fc00c2c3ecb2ed39c70df887db50e649dc0ad1e5ee12a270a4320771604429d#rd


IndexError: list index out of range

In [77]:
run(258, 259)

https://mp.weixin.qq.com/s?__biz=MzI0NDcxNzc5Mg==&mid=2247485273&idx=2&sn=7d5c50da130a60612f9827798de921a3&chksm=e958c6d6de2f4fc00c2c3ecb2ed39c70df887db50e649dc0ad1e5ee12a270a4320771604429d#rd
Title:  NCE3 | Lesson 3 An unknown goddess　无名女神
write in to file: http://y.biomooc.com/wangjl/en20181009.html
Not exist the date
commit successfully
258 4.7712892922093255


In [25]:
run(259)

https://mp.weixin.qq.com/s?__biz=MzI0NDcxNzc5Mg==&mid=2247485271&idx=1&sn=5e2bf882c3f3a20fbf57bacdfb7f4513&chksm=e958c6d8de2f4fcecefed98047ec02f44b5b290b7aa81a4fbe10425ffe438480930516286a00#rd
Title:  NPR|China Reshapes The Vital Mekong River To Power Its Expansion
write in to file: http://y.biomooc.com/wangjl/en20181008.html
Not exist the date
commit successfully
259 4.704945065253615
https://mp.weixin.qq.com/s?__biz=MzI0NDcxNzc5Mg==&mid=2247485271&idx=2&sn=9dac95273942fca51a9a935173b77563&chksm=e958c6d8de2f4fce69aac926076d55feb9416558ff40a4289e8ff768a3815183bec3cf1d2604#rd


IndexError: list index out of range

In [78]:
run(260, 261)

https://mp.weixin.qq.com/s?__biz=MzI0NDcxNzc5Mg==&mid=2247485271&idx=2&sn=9dac95273942fca51a9a935173b77563&chksm=e958c6d8de2f4fce69aac926076d55feb9416558ff40a4289e8ff768a3815183bec3cf1d2604#rd
Title:  NCE3|Lesson 2 Thirteen equals one 十三等于一
write in to file: http://y.biomooc.com/wangjl/en20181008.html
Not exist the date
commit successfully
260 4.993372067134725


In [26]:
run(261)

https://mp.weixin.qq.com/s?__biz=MzI0NDcxNzc5Mg==&mid=2247485256&idx=1&sn=2b6217043a9f305c0e51d2cc3bf13eba&chksm=e958c6c7de2f4fd19c36c8d6e791570a7f51b18dd804a2398af9c375463ac0f8027ead05fbf7#rd
Title:  psychologyToday|The Bias and Embarrassment of Hypocrisy
write in to file: http://y.biomooc.com/wangjl/en20181007.html
Not exist the date
commit successfully
261 3.969717090489096
https://mp.weixin.qq.com/s?__biz=MzI0NDcxNzc5Mg==&mid=2247485256&idx=2&sn=89587ad37cd0eafad02b15c0c1237718&chksm=e958c6c7de2f4fd1c014d45d6833522638cb5c2e65ba0bfebdc1447582b5d9036a76112b4d2d#rd


IndexError: list index out of range

In [79]:
run(262, 263)

https://mp.weixin.qq.com/s?__biz=MzI0NDcxNzc5Mg==&mid=2247485256&idx=2&sn=89587ad37cd0eafad02b15c0c1237718&chksm=e958c6c7de2f4fd1c014d45d6833522638cb5c2e65ba0bfebdc1447582b5d9036a76112b4d2d#rd
Title:  NCE3 | Lesson 1 A Puma at large 逃遁的美洲狮
write in to file: http://y.biomooc.com/wangjl/en20181007.html
Not exist the date
commit successfully
262 3.183020286508987


In [27]:
run(263)

https://mp.weixin.qq.com/s?__biz=MzI0NDcxNzc5Mg==&mid=2247485249&idx=1&sn=b47ddc946213c56b8b4cb5da4bbc4a68&chksm=e958c6cede2f4fd802cf7f17232e3dbe9829b2310602ac962b657dc90d4da2ee6b22905ef87a#rd
Title:  science|‘Revolution based on ...’ honored with chemistry Nobel
write in to file: http://y.biomooc.com/wangjl/en20181006.html
Not exist the date
commit successfully
263 3.0883070583390966
https://mp.weixin.qq.com/s?__biz=MzI0NDcxNzc5Mg==&mid=2247485244&idx=1&sn=2eca71ac61efd2e66ba69123cd358920&chksm=e958c6b3de2f4fa599e438562c7041801dd61d2ee5f7299046ffe0e71a11ab3c5ec36c8e49ff#rd
Title:  TIME|How Amazon&#39;s Growing ... Is Forcing It to Boost Worker Pay
write in to file: http://y.biomooc.com/wangjl/en20181005.html
Not exist the date
commit successfully
264 3.4619714337285554
https://mp.weixin.qq.com/s?__biz=MzI0NDcxNzc5Mg==&mid=2247485239&idx=1&sn=a1ab631c0d27b7e2b727583ffa0232da&chksm=e958c6b8de2f4faea5094e99827c4e392bb449627c9d726a2f2033cc55d9e861e8498ea6daae#rd
Title:  Science|Cancer imm

IndexError: list index out of range

In [80]:
run(266, 267)

https://mp.weixin.qq.com/s?__biz=MzI0NDcxNzc5Mg==&mid=2247485239&idx=2&sn=8846604b1ccbcafd8180246002f9b3e4&chksm=e958c6b8de2f4fae4fbb06b457740744fe106d1f0fec340c803fdf4f8676ca03635c6f7cef02#rd
Title:  英语读报 | 怎么在一个月内让英语听力有明显的提高？刻意练习法
write in to file: http://y.biomooc.com/wangjl/en20181004.html
Not exist the date
commit successfully
266 3.5180690784216244


In [28]:
run(267)

https://mp.weixin.qq.com/s?__biz=MzI0NDcxNzc5Mg==&mid=2247485232&idx=1&sn=d6824bd9ac1f86f4f5febbac65f41e57&chksm=e958c6bfde2f4fa9af1c8faaa6df97594811c7c6dde91bb48aee7f21654f6e3e938725522aab#rd
Title:  FT | US considered ban on student visas for Chinese nationals
write in to file: http://y.biomooc.com/wangjl/en20181003.html
Not exist the date
commit successfully
267 4.847554109469944
https://mp.weixin.qq.com/s?__biz=MzI0NDcxNzc5Mg==&mid=2247485226&idx=1&sn=98ceb18b5aee060c9371fa3ae3356a1c&chksm=e958c6a5de2f4fb38bd5bc4647470932477cdf433ed4afbb66216310886679d102e663b52879#rd
Title:  economist|Artificial intelligence: The gladiator’s edge
write in to file: http://y.biomooc.com/wangjl/en20181002.html
Not exist the date
commit successfully
268 3.0974929110288825
https://mp.weixin.qq.com/s?__biz=MzI0NDcxNzc5Mg==&mid=2247485221&idx=1&sn=0ef7fc2fad43281263aeee49fe002380&chksm=e958c6aade2f4fbc695af78d6cd7704fd0e123280d122100be061eab392c6b8b6696e764ac15#rd
Title:  economist |Russia: A sea of trou

KeyError: ', 2'

In [82]:
run(275, 276)

https://mp.weixin.qq.com/s?__biz=MzI0NDcxNzc5Mg==&mid=2247485195&idx=2&sn=cef62c3dd8dde8fb6748d906b07aa978&chksm=e958c684de2f4f92cc5cf747e0cb483e45b9c94f86292304994d513e65f02c49ef5ba31259c7#rd
Title:  NG|Why Koalas Are Suddenly Drinking Extra Water
write in to file: http://y.biomooc.com/wangjl/en20180926.html
Not exist the date
commit successfully
275 4.303926805293925


In [39]:
run(275)

https://mp.weixin.qq.com/s?__biz=MzI0NDcxNzc5Mg==&mid=2247485195&idx=2&sn=cef62c3dd8dde8fb6748d906b07aa978&chksm=e958c684de2f4f92cc5cf747e0cb483e45b9c94f86292304994d513e65f02c49ef5ba31259c7#rd
day= 26
Title:  NG|Why Koalas Are Suddenly Drinking Extra Water
write in to file: http://y.biomooc.com/wangjl/en20180926.html
The date 20180926 has already existed.
275 4.162507596791157
https://mp.weixin.qq.com/s?__biz=MzI0NDcxNzc5Mg==&mid=2247485194&idx=1&sn=3d484220ff1cb1bb28eac77dacfaac2b&chksm=e958c685de2f4f93c07a626392c08f689f7bf37b291cacdb40589cffbfd94c1ad6255e9001e3#rd
day= 25
Title:  Guardian|Solving the genome puzzle
write in to file: http://y.biomooc.com/wangjl/en20180925.html
Not exist the date
commit successfully
276 4.433985553705874
https://mp.weixin.qq.com/s?__biz=MzI0NDcxNzc5Mg==&mid=2247485190&idx=1&sn=9c791ffdb04109623d9b7eed2b722455&chksm=e958c689de2f4f9fff30d1c67f236c885d4bf3db5551891181d05dcc9bd1d6db6c0d3668eea3#rd
day= 24
Title:  economist|Education: Class struggle
write in

day= 06
Title:  britishCouncil | Making time for science
write in to file: http://y.biomooc.com/wangjl/en20180906.html
Not exist the date
commit successfully
295 4.793627251510168
https://mp.weixin.qq.com/s?__biz=MzI0NDcxNzc5Mg==&mid=2247485104&idx=1&sn=5c406ab4bbc47abe8f8c047330aa311b&chksm=e958c73fde2f4e29080fc3466e040209e926643a6fc06c74bc9ef50ea1648ed5dfcf3e33def0#rd
day= 05
Title:  foxNews|Liu QD arrested in M.. for alleged sexual misconduct
write in to file: http://y.biomooc.com/wangjl/en20180905.html
Not exist the date
commit successfully
296 4.781095960262929
https://mp.weixin.qq.com/s?__biz=MzI0NDcxNzc5Mg==&mid=2247485100&idx=1&sn=e0f89428748fbbfb423e18e3519c2924&chksm=e958c723de2f4e350ef598da8f4d66164223cfe2c037df6c9d2fc4cdd95cd3f9bb08005ec65f#rd
day= 04
Title:  Reuters|Cryptoeconomics:Can blockchain reinvent justice systems?
write in to file: http://y.biomooc.com/wangjl/en20180904.html
Not exist the date
commit successfully
297 4.0173987458825895
https://mp.weixin.qq.com/s?__

https://mp.weixin.qq.com/s?__biz=MzI0NDcxNzc5Mg==&mid=2247485012&idx=1&sn=f530d0fa3c29d4265d150559a3c14633&chksm=e958c7dbde2f4ecd45a36cb6da18203e04dd7e3aabe684da225408577b67bb51488eddb8f2ab#rd
day= 16
Title:  economist | Silicon Valley gets queasy about Chinese money
write in to file: http://y.biomooc.com/wangjl/en20180816.html
Not exist the date
commit successfully
316 4.861891093214172
https://mp.weixin.qq.com/s?__biz=MzI0NDcxNzc5Mg==&mid=2247485008&idx=1&sn=113b6fdc764deb319b6effc6256820e7&chksm=e958c7dfde2f4ec98e9f949066880d1a85c59350491d4acf36e984e827e4d864e75661f3efb7#rd
day= 15
Title:  newsweek | U.S. SOYBEAN SHIP THAT FLOATED NEAR CHINA FOR A ...
write in to file: http://y.biomooc.com/wangjl/en20180815.html
Not exist the date
commit successfully
317 3.390790420142058
https://mp.weixin.qq.com/s?__biz=MzI0NDcxNzc5Mg==&mid=2247485004&idx=1&sn=cbdfc858bb7d541527cfa40e20dfad9a&chksm=e958c7c3de2f4ed5ba878588a38689ed5b886f2a3776744ece2566b0898a9ef7fdf7ea04d92c#rd
day= 13
Title:  NPR|C

IndexError: list index out of range

In [90]:
run(321, 322)

https://mp.weixin.qq.com/s?__biz=MzI0NDcxNzc5Mg==&mid=2247484994&idx=1&sn=ed789902ead84cbf94abdf5c7a66102c&chksm=e958c7cdde2f4edb262e9c70b20d9604bc0b34656731b897ff1fb491e1f974b20947ca55d957#rd
Title:  HWR | &#39;Westworld&#39;: What the S2 Finale&#39;s Post-Credits Scene Means
write in to file: http://y.biomooc.com/wangjl/en20180811.html
Not exist the date
commit successfully
321 3.0671326946232265


In [47]:
run(326)

https://mp.weixin.qq.com/s?__biz=MzI0NDcxNzc5Mg==&mid=2247484972&idx=1&sn=60caff3f031f3e33abf23db1455e1f8d&chksm=e958c7a3de2f4eb58b3eb123916585bf87456aca15b75d55153660998f55f52240abee706140#rd
Title:  nypost|Trump keeps up attacks against media: ‘They can also ...
write in to file: http://y.biomooc.com/wangjl/en20180806.html
Not exist the date
commit successfully
326 4.624552685288344
https://mp.weixin.qq.com/s?__biz=MzI0NDcxNzc5Mg==&mid=2247484968&idx=1&sn=a0f6c48a38e5904b1d31e2bfe2398d68&chksm=e958c7a7de2f4eb1b9fe45fe2e666e243f3fcb44182a1d28b9c32500f0f49da6617eb850c816#rd
                    

                    

                    
                    
                    Aug 05, 2018 | 254 wordsby Paul R. La Monica&nbsp; &nbsp;@lamo
Title:  CNN | Apple reaches $1,000,000,000,000 value
write in to file: http://y.biomooc.com/wangjl/en20180805.html
Not exist the date
commit successfully
327 4.71635406737202
https://mp.weixin.qq.com/s?__biz=MzI0NDcxNzc5Mg==&mid=2247484967&idx=1&sn=e

https://mp.weixin.qq.com/s?__biz=MzI0NDcxNzc5Mg==&mid=2247484904&idx=1&sn=052f16675dbad5fa9046c09cc81cc6e0&chksm=e958c467de2f4d71e4648ee19803225d287068297fded2dbd7ca6bbd0af2302cf1e813f4972b#rd
                    

                    

                    
                    
                    Jul 21, 2018 | 3240 wordsA Critic at Large|July 23, 2018 Issue
Title:  newYorker|Can Economists and Humanists Ever Be Friends?
write in to file: http://y.biomooc.com/wangjl/en20180721.html
Not exist the date
commit successfully
342 4.385286988520714
https://mp.weixin.qq.com/s?__biz=MzI0NDcxNzc5Mg==&mid=2247484903&idx=1&sn=4ada36d75bb42f470577027bb6048da0&chksm=e958c468de2f4d7eb6db6873d4b79f166770d56f2aa1ccc82001ea5c340a1ad8d68c76aa9880#rd
                    

                    

                    
                    
                    Jul 20, 2018 | 544 wordsIt's Nearly Impossible to Diagnose Alzheimer's Disease i
Title:  Time|It&#39;s Nearly Impossible to Diagnose Alzheimer&#39;s Dis

Title:  名著导读 | 圣经-创世纪 中英文对照选读
write in to file: http://y.biomooc.com/wangjl/en20180708.html
The date 20180708 has already existed.
356 4.16774278900483
https://mp.weixin.qq.com/s?__biz=MzI0NDcxNzc5Mg==&mid=2247484848&idx=1&sn=1041a1cb2266f2520c8619c860239d68&chksm=e958c43fde2f4d29ca5920e5e46419308e97593760139a1181c63c5d9efed279f09733338a47#rd
                    

                    

                    
                    
                    Jul 07, 2018 | 903 wordsEmerging technologies in speech generation raise ethics and security concerns</stro
Title:  sciAmerican|New AI Tech Can Mimic Any Voice
write in to file: http://y.biomooc.com/wangjl/en20180707.html
Not exist the date
commit successfully
357 4.635394187110202
https://mp.weixin.qq.com/s?__biz=MzI0NDcxNzc5Mg==&mid=2247484848&idx=2&sn=1bc49d4d58a7a894e7925838e3bdbe25&chksm=e958c43fde2f4d299226b1cd6cee38419eacd5f26ec3df347b31b5f4e088251bf034cb73e00d#rd
                    

                    

                    
        

https://mp.weixin.qq.com/s?__biz=MzI0NDcxNzc5Mg==&mid=2247484771&idx=1&sn=063846bbd5e1f1623f03c2a54d9e639d&chksm=e958c4ecde2f4dfac31710e760fbe72151e75be50d40cc5ae8792bf002df5805c263293a9a96#rd
                    

                    

                    
                    
                    Jun 25, 2018 | 3562 wordsCan a Parent’s Life Experience Change the Genes a Child
Title:  Atlantic|Can a Parent’s Life Experience Change the Genes a ...
write in to file: http://y.biomooc.com/wangjl/en20180625.html
Not exist the date
commit successfully
372 3.804844141342781
https://mp.weixin.qq.com/s?__biz=MzI0NDcxNzc5Mg==&mid=2247484767&idx=1&sn=5df11592fb7d442e44466f50b3994840&chksm=e958c4d0de2f4dc6ba1a2789390de925dc810e4de93289576bb1ccfd98ac001e9f11328c0f03#rd
                    

                    

                    
                    
                    Jun 24, 2018 | 1044 words20180623雅思话题:&nbsp;罗马饮食与宴会
Title:  npr|What Did Ancient Romans Eat? New Novel Serves Up Meals ...
writ

                    

                    

                    
                    
                    Jun 12, 2018 | 1130 wordsTrump Strikes Deal to Save China’s ZTE as North Korea M
Title:  Nytimes|Trump Strikes Deal to Save ZTE as N K Meeting Looms
write in to file: http://y.biomooc.com/wangjl/en20180612.html
Not exist the date
commit successfully
387 3.686658968874654
https://mp.weixin.qq.com/s?__biz=MzI0NDcxNzc5Mg==&mid=2247484687&idx=1&sn=ed9743be71a5de6a044200c1d632cde8&chksm=e958c480de2f4d96d82ce7dd892f121d0552fec588a23e4912d3900a2e5ca12ff732c2f7c3ad#rd
                    

                    

                    
                    
                    Jun 11, 2018 | 947 wordsThrough the G7 summit, the brittle pretense of unity held together. T
Title:  Atlantic|Trump Goes to War Against the Democracies
write in to file: http://y.biomooc.com/wangjl/en20180611.html
Not exist the date
commit successfully
388 4.182364560721194
https://mp.weixin.qq.com/s?__biz=MzI0NDcxNzc5Mg

Title:  investing|U.S. to continue trade actions against China: White...
write in to file: http://y.biomooc.com/wangjl/en20180530.html
The date 20180530 has already existed.
402 3.0941720297338273
https://mp.weixin.qq.com/s?__biz=MzI0NDcxNzc5Mg==&mid=2247484620&idx=1&sn=06c714685b255f3c2f8250feebbf6c19&chksm=e958c543de2f4c55d740841d1758b0b8ea46a960f9fd695ca5676939ca9d93c24c9762ddc6f0#rd
                    

                    

                    
                    
                    Heidi Ledford|May 28, 2018 | 646 wordsRush to protect lucra


ValueError: invalid literal for int() with base 10: 'May 28, 2018'

In [86]:
run(403, 404, True)

https://mp.weixin.qq.com/s?__biz=MzI0NDcxNzc5Mg==&mid=2247484620&idx=1&sn=06c714685b255f3c2f8250feebbf6c19&chksm=e958c543de2f4c55d740841d1758b0b8ea46a960f9fd695ca5676939ca9d93c24c9762ddc6f0#rd
Title:  Nature|Rush to protect ... antibody patents kicks into gear
write in to file: http://y.biomooc.com/wangjl/en20180529.html
Not exist the date
commit successfully
403 3.048608908425566


In [48]:
run(404)

https://mp.weixin.qq.com/s?__biz=MzI0NDcxNzc5Mg==&mid=2247484616&idx=1&sn=04a353fb595f972a280d2bbe1f57a502&chksm=e958c547de2f4c51bdea1b1bc362eb8b21ba206d4c513ab2d833e26490824903eb10ba53e6f8#rd
                    

                    

                    
                    
                    May 28, 2018 | 967 wordsFor the Communist Party, this could be a risk<p
Title:  Economist|China wages war on apps offering news and jokes
write in to file: http://y.biomooc.com/wangjl/en20180528.html
Not exist the date
commit successfully
404 3.6712567315687625
https://mp.weixin.qq.com/s?__biz=MzI0NDcxNzc5Mg==&mid=2247484615&idx=1&sn=98d609b0dfb707fdc2af36ffaaac165f&chksm=e958c548de2f4c5ec59f0425c5bdb8a4abde2f107cd667060674ffa73aff67678d927b6387db#rd
                    

                    

                    
                    
                    &nbsp;May 27, 2018 | 799 words<span style


KeyError: '&nb'

In [52]:
run(405)

https://mp.weixin.qq.com/s?__biz=MzI0NDcxNzc5Mg==&mid=2247484615&idx=1&sn=98d609b0dfb707fdc2af36ffaaac165f&chksm=e958c548de2f4c5ec59f0425c5bdb8a4abde2f107cd667060674ffa73aff67678d927b6387db#rd
Title:  HBR|Technical Experts Need to Get Better at Telling Stories
write in to file: http://y.biomooc.com/wangjl/en20180527.html
Not exist the date
commit successfully
405 4.829779600547725
https://mp.weixin.qq.com/s?__biz=MzI0NDcxNzc5Mg==&mid=2247484614&idx=1&sn=92f24d8e0365083054a663f45c67b389&chksm=e958c549de2f4c5f3e2173c4cc663609abf6664337dc30480696ca86cf00b62d8c2c1a3b2dda#rd
Title:  CNN | Trump cancels Singapore summit in letter to Kim Jong Un
write in to file: http://y.biomooc.com/wangjl/en20180526.html
Not exist the date
commit successfully
406 3.437388550161918
https://mp.weixin.qq.com/s?__biz=MzI0NDcxNzc5Mg==&mid=2247484606&idx=1&sn=20e6a6f6f895ff8e7bad181234c0e8e0&chksm=e958c531de2f4c27bd343fc065b69e238fb753bbbccbca5f7262e96284350a7ad907bcbd1fe4#rd
Title:  Economist|Why many rich Chine

ValueError: invalid literal for int() with base 10: 'May 22, 2018'

In [87]:
run(409, 410, True)

https://mp.weixin.qq.com/s?__biz=MzI0NDcxNzc5Mg==&mid=2247484601&idx=1&sn=b9d5f45165657ec99106e2c6ac66a428&chksm=e958c536de2f4c2040e9f6f131b40f12e0b38b58660ef6750fe4c754f75b00e63b7e9fbe7ad2#rd
Title:  TheWeek | How China and Wall Street won Trump&#39;s trade war
write in to file: http://y.biomooc.com/wangjl/en20180523.html
Not exist the date
commit successfully
409 3.9748658792427514


In [54]:
run(410)

https://mp.weixin.qq.com/s?__biz=MzI0NDcxNzc5Mg==&mid=2247484583&idx=1&sn=cf4e12295cc5702b9c880235477fffb4&chksm=e958c528de2f4c3ee1e2bf0d2c3dfb5b2f5003d7ddfc20a00741aa6c74eb612f458b0dc2844a#rd
Title:  abc|US, China putting trade war on hold after progress in talks
write in to file: http://y.biomooc.com/wangjl/en20180522.html
Not exist the date
commit successfully
410 4.852561237510426
https://mp.weixin.qq.com/s?__biz=MzI0NDcxNzc5Mg==&mid=2247484583&idx=2&sn=b74ab358edfe0676880c3370de1270bd&chksm=e958c528de2f4c3e6f5cb5cf5c6f6e5e40f315b811124e157508b093c181d2893679643ef1b8#rd
Title:  FT|US detente with China postpones trade war but ...
write in to file: http://y.biomooc.com/wangjl/en20180522.html
The date 20180522 has already existed.
411 4.743190014950936
https://mp.weixin.qq.com/s?__biz=MzI0NDcxNzc5Mg==&mid=2247484577&idx=1&sn=2b074d97b878640ade5ecf59d3300815&chksm=e958c52ede2f4c3852635eaccdb29887f5380bbb09bd36e65843f5b4278afcfcc1ec748306a7#rd
Title:  Historians|Why Study History? (199

https://mp.weixin.qq.com/s?__biz=MzI0NDcxNzc5Mg==&mid=2247484486&idx=1&sn=2bfdd430b65ac9eceb2dd27d03b4b305&chksm=e958c5c9de2f4cdf4d33f2bc97d2fe233ed2a9289e56252084edff8a3254c285d324100e04ca#rd
Title:  Atlantic|&#39;Infinity War&#39; and the Point of Comic-Book Deaths
write in to file: http://y.biomooc.com/wangjl/en20180503.html
Not exist the date
commit successfully
431 4.645882176951836
https://mp.weixin.qq.com/s?__biz=MzI0NDcxNzc5Mg==&mid=2247484482&idx=1&sn=689c700ab35ed67b3b023bb78be4b8e7&chksm=e958c5cdde2f4cdb78254c969fab76297727dfd661cd3c5120bfc8cd6cbfa8af3da035f9332d#rd
Title:  BBC|Is Tesla heading for trouble?
write in to file: http://y.biomooc.com/wangjl/en20180502.html
Not exist the date
commit successfully
432 4.43161437839556
https://mp.weixin.qq.com/s?__biz=MzI0NDcxNzc5Mg==&mid=2247484478&idx=1&sn=d26f7630b6639d7912a74723d102da6d&chksm=e958c5b1de2f4ca7d3f0c8dd20783f218423fa817b29ccedd3ad63614a79f0ba57427f0e623e#rd
Title:  Dataversity|A Brief History of Artificial Intellige

Title:  guardian|England&#39;s forests: a brief history of trees
write in to file: http://y.biomooc.com/wangjl/en20180417.html
Not exist the date
commit successfully
451 4.673271026674954
https://mp.weixin.qq.com/s?__biz=MzI0NDcxNzc5Mg==&mid=2247484402&idx=1&sn=42d7aa1d9723802a882874fa114f460e&chksm=e958c27dde2f4b6bd57517e4a2dfcc51faa923e4bb9ada146653e7237ec9b15495825ed387d0#rd
Title:  rfs | A brief history of British woodlands
write in to file: http://y.biomooc.com/wangjl/en20180416.html
Not exist the date
commit successfully
452 4.429997723898996
https://mp.weixin.qq.com/s?__biz=MzI0NDcxNzc5Mg==&mid=2247484401&idx=1&sn=e6f1e52ee21f0e20537fdd927315057c&chksm=e958c27ede2f4b68f0055d69c84c220eec6571173bedec2d5235cececc2332132ab4219278b9#rd
Title:  BI|&#39;Westworld&#39; season 2 is even better than the first and ...
write in to file: http://y.biomooc.com/wangjl/en20180415.html
Not exist the date
commit successfully
453 4.413323061184869
https://mp.weixin.qq.com/s?__biz=MzI0NDcxNzc5Mg==&m

Title:  Guardian|Elephant &#39;smoking&#39; footage baffles experts
write in to file: http://y.biomooc.com/wangjl/en20180328.html
Not exist the date
commit successfully
472 3.2349081067588994
https://mp.weixin.qq.com/s?__biz=MzI0NDcxNzc5Mg==&mid=2247484300&idx=1&sn=32db43579d9ad69670baee6fbf19745c&chksm=e958c203de2f4b15aa1bb133ff00ee2d4ebd17befdee561e2884a368a702e091b41de4690949#rd
Title:  Reuters|US asks China not to implement ban on foreign garbage
write in to file: http://y.biomooc.com/wangjl/en20180327.html
Not exist the date
commit successfully
473 3.8427377669325895
https://mp.weixin.qq.com/s?__biz=MzI0NDcxNzc5Mg==&mid=2247484296&idx=1&sn=97dc81343fe5e639765c475c144ef919&chksm=e958c207de2f4b11ba4de4fc7788d6f06719f650de05f317917e552121aee679fc45815cc2a1#rd
Title:  CNBC|Facebook saves extensive data made by Android users
write in to file: http://y.biomooc.com/wangjl/en20180326.html
Not exist the date
commit successfully
474 4.515667879614426
https://mp.weixin.qq.com/s?__biz=MzI0NDc

IndexError: list index out of range

In [ ]:
# 484-485 被删除

In [56]:
run(486)

https://mp.weixin.qq.com/s?__biz=MzI0NDcxNzc5Mg==&mid=2247484250&idx=1&sn=7a8de89b1541e13aa83efffbddcf10c7&chksm=e958c2d5de2f4bc376dfec272009247f35bf07ed2914c7ce2a39862a0508322cc3367f6a2d22#rd
Title:  Guardian|We need to use gene editing wisely but also embrace ...
write in to file: http://y.biomooc.com/wangjl/en20180317.html
Not exist the date
commit successfully
486 3.0626626282093623
https://mp.weixin.qq.com/s?__biz=MzI0NDcxNzc5Mg==&mid=2247484249&idx=1&sn=4374775b46efea65383eae16fa109ae0&chksm=e958c2d6de2f4bc053ec4a892b6405d49dd048eafa17401aefecad8aa13e5f2bf52ef9b6939a#rd
Title:  HBR|Making Better Use of Health Care Data
write in to file: http://y.biomooc.com/wangjl/en20180316.html
Not exist the date
commit successfully
487 4.980578396225798
https://mp.weixin.qq.com/s?__biz=MzI0NDcxNzc5Mg==&mid=2247484248&idx=1&sn=6f867d7b52beff44390a87ecafb7f506&chksm=e958c2d7de2f4bc1135d49a7ec1a2c07b135439e7ec660c2ea27fd3e153e8a2acce10374200e#rd
Title:  Guardian|Stephen Hawking, science&#39;s bri

IndexError: list index out of range

In [ ]:
# 496 被删除

In [57]:
run(497)

https://mp.weixin.qq.com/s?__biz=MzI0NDcxNzc5Mg==&mid=2247484197&idx=1&sn=b5379da79972bfe47e556b92fee7e930&chksm=e958c2aade2f4bbc3713a59eace71799fa9ea7731e646573864ef62689fe1d6141da40e0a9d7#rd
Title:  BBC|Baby bird fossil is &#39;rarest of the rare&#39;
write in to file: http://y.biomooc.com/wangjl/en20180307.html
Not exist the date
commit successfully
497 3.6434322118739466
https://mp.weixin.qq.com/s?__biz=MzI0NDcxNzc5Mg==&mid=2247484192&idx=1&sn=f16c6d10387e5b4b0cae1b75472d10a6&chksm=e958c2afde2f4bb9c9115d71db741393bb3bd7f48a659cd0d5b1e114f6674aff7b0c2642024c#rd
Title:  Guardian|The DNA database..is key to beating our rarest diseases
write in to file: http://y.biomooc.com/wangjl/en20180226.html
Not exist the date
commit successfully
498 4.77168663545159
https://mp.weixin.qq.com/s?__biz=MzI0NDcxNzc5Mg==&mid=2247484191&idx=1&sn=fe44ab73547c839ac923ad1de4938ca2&chksm=e958c290de2f4b86b071d80ce5135d64a4ab72ae4dd180bb26d488a291446ce6a1347d6f492c#rd
Title:  economist|OPEC mulls a long-term 

Title:  science|Make replication studies an essential part of science
write in to file: http://y.biomooc.com/wangjl/en20180121.html
Not exist the date
commit successfully
517 4.197974291493949
https://mp.weixin.qq.com/s?__biz=MzI0NDcxNzc5Mg==&mid=2247484109&idx=1&sn=e7f70727db5aa9b01253413cabe5b390&chksm=e958c342de2f4a5459c57f4b1004472996ed23db85eccb78f3183367cd29b7f551609f6f63b4#rd
Title:  wired|Menacing malware shows the dangers of ... system sabotage
write in to file: http://y.biomooc.com/wangjl/en20180120.html
Not exist the date
commit successfully
518 3.453044500724876
https://mp.weixin.qq.com/s?__biz=MzI0NDcxNzc5Mg==&mid=2247484103&idx=1&sn=ce8fbfb8edda12d5b662c3fa48611e6c&chksm=e958c348de2f4a5ee80fb201522462c4c7e0a69e969e55dd30a5d7915d4e034fe3354499b066#rd
Title:  usatoday|Earth&#39;s magnetic shift to cost Wichita airport
write in to file: http://y.biomooc.com/wangjl/en20180119.html
Not exist the date
commit successfully
519 4.025718895206668
https://mp.weixin.qq.com/s?__biz=Mz

https://mp.weixin.qq.com/s?__biz=MzI0NDcxNzc5Mg==&mid=2247484013&idx=1&sn=c05024ae6e3e49c87d93ec874c1f7114&chksm=e958c3e2de2f4af4622852d8a8f3ba80a9cfa86d2628831edf1224c07db7b4b56aca994c2ddb#rd
Title:  [Guardian]Ice will return but extinctions can&#39;t be reversed
write in to file: http://y.biomooc.com/wangjl/en20171229.html
Not exist the date
commit successfully
538 4.580176864331073
https://mp.weixin.qq.com/s?__biz=MzI0NDcxNzc5Mg==&mid=2247484009&idx=1&sn=0c77f79f3515606f10994e051e0f1182&chksm=e958c3e6de2f4af0c03641709fce1f75a5414539815e7c9629ab45b49939da1952d35d4baab4#rd
Title:  [BI]How birth order affects your chances of success
write in to file: http://y.biomooc.com/wangjl/en20171228.html
Not exist the date
commit successfully
539 3.451776231559884
https://mp.weixin.qq.com/s?__biz=MzI0NDcxNzc5Mg==&mid=2247484005&idx=1&sn=19572fb51fc58dbe731373d60cc4e58f&chksm=e958c3eade2f4afc83a80e817693aaa8b8e1b96d24dd2797403d1ba89d692a4757157c8c8d21#rd
Title:  [BBC]Why are China instant noodle s

https://mp.weixin.qq.com/s?__biz=MzI0NDcxNzc5Mg==&mid=2247483933&idx=1&sn=148e42129561cd2cbcd4f89d6888f993&chksm=e958c392de2f4a84f7c27df227101c7f08c7a622832ea2c149a005be2c420678b87bc894dfe2#rd
Title:  [BBC]Yellowstone
write in to file: http://y.biomooc.com/wangjl/en20171207.html
Not exist the date
commit successfully
559 4.540943171676622
https://mp.weixin.qq.com/s?__biz=MzI0NDcxNzc5Mg==&mid=2247483932&idx=1&sn=c5d7463bac2eafbb9fbc038934afb880&chksm=e958c393de2f4a852490a8ea798415aeb8ccc3dcc407035bc5b174c711fbe585332ce325177c#rd
Title:  Job Interview Answers Guide
write in to file: http://y.biomooc.com/wangjl/en20171206.html
Not exist the date
commit successfully
560 4.902202696437872
https://mp.weixin.qq.com/s?__biz=MzI0NDcxNzc5Mg==&mid=2247483922&idx=1&sn=2a2685137c4619f16c81a93455675b47&chksm=e958c39dde2f4a8b9dae4986ad12bfac7a3790702eec3002786854bcfb40ec12308cb59a27ac#rd
Title:  [nbcNews]Liu Yifei cast as lead in live-action remake of &#39;Mulan&#39;
write in to file: http://y.biomoo

IndexError: list index out of range

In [ ]:
# 578 被删除

In [58]:
run(579)

https://mp.weixin.qq.com/s?__biz=MzI0NDcxNzc5Mg==&mid=2247483848&idx=1&sn=6e2d9c77497bc73b91ca920d56a15be0&chksm=e958c047de2f49516079e400e79b4d27d9abcfa8b60e44b8a8eb68ae7c916fbf359a7cbd7c90#rd
Title:  [wikihow]wikiHow to Prepare for IELTS
write in to file: http://y.biomooc.com/wangjl/en20171118.html
Not exist the date
commit successfully
579 3.5711951485848763
https://mp.weixin.qq.com/s?__biz=MzI0NDcxNzc5Mg==&mid=2247483844&idx=1&sn=d7172490cbe80ea03c63ba0ba9417235&chksm=e958c04bde2f495dd777573eb11541a958d4adffc8a7a63cd0540e38a44bbb84bf181a992010#rd
Title:  [inc]How to Become the Best in the World at What You Do
write in to file: http://y.biomooc.com/wangjl/en20171117.html
Not exist the date
rollback
(<class 'pymysql.err.DataError'>, DataError(1406, "Data too long for column 'msg_content' at row 1"), <traceback object at 0x000002E9632D1F48>)
580 3.4745923437857513
https://mp.weixin.qq.com/s?__biz=MzI0NDcxNzc5Mg==&mid=2247483844&idx=2&sn=e71c00b84d94c4d90729ca4aee475428&chksm=e958c04bde

https://mp.weixin.qq.com/s?__biz=MzI0NDcxNzc5Mg==&mid=2247483761&idx=1&sn=f02b34c1d8e4565fe4f5c987a1ece8f7&chksm=e958c0fede2f49e8f540e7f01a81c1a11bfdaca6cf380aa58c13933b417b2775eb517c5292a0#rd
Title:  [vanityfair]WILL WE EVER RETURN TO NORMAL AFTER TRUMP?
write in to file: http://y.biomooc.com/wangjl/en20171010.html
Not exist the date
commit successfully
600 3.4024737898193926
https://mp.weixin.qq.com/s?__biz=MzI0NDcxNzc5Mg==&mid=2247483760&idx=1&sn=3cc5d70c96669f59c16a84b76520fba9&chksm=e958c0ffde2f49e93ceb1a9b8b7dbf3b40286c367f56465c6921909b18dbde87b708a075ee99#rd
Title:  [nature]China’s bid to be a DNA superpower
write in to file: http://y.biomooc.com/wangjl/en20171009.html
Not exist the date
commit successfully
601 3.0333032533727344
https://mp.weixin.qq.com/s?__biz=MzI0NDcxNzc5Mg==&mid=2247483750&idx=1&sn=178cceabfbf04636b511679228fff3b8&chksm=e958c0e9de2f49ff6401dd6c587eeb82e4294fd92aef22988b9a96baf02ff57310d178d6e2d9#rd
Title:  [sci60s]Nobel in Chemistry for Seeing Biomolecules 

https://mp.weixin.qq.com/s?__biz=MzI0NDcxNzc5Mg==&mid=2247483661&idx=1&sn=a08c6cf89d5c1c231893aa8496130963&chksm=e958c082de2f49944eba285ef599834907b5541c412af4e43566cbfc7886b0b61d2865bebcb9#rd
Title:  Fate of Universe revealed by galactic lens
write in to file: http://y.biomooc.com/wangjl/en20170917.html
Not exist the date
commit successfully
621 4.667998343435272
https://mp.weixin.qq.com/s?__biz=MzI0NDcxNzc5Mg==&mid=2247483656&idx=1&sn=b66f6c753b776b3b68f9f1e21704fd8a&chksm=e958c087de2f49915b02f1580f7c97f637ed62198cad7bdfa2203b5f26526153918f3bfb11b1#rd
Title:  The beauty of journey is found not in the destination
write in to file: http://y.biomooc.com/wangjl/en20170511.html
Not exist the date
commit successfully
622 4.3987099117748185
https://mp.weixin.qq.com/s?__biz=MzI0NDcxNzc5Mg==&mid=2247483653&idx=1&sn=19dfcc53bb68b6ca7b2ff5eb07461972&chksm=e958c08ade2f499c65419469d7f2fa4738d12f4e47f5c738fc78e39b76e796765a3f1c493034#rd
Title:  005words-rawFull-Illumina-GitHub-Supermoon-repository

In [92]:
run(580, 581, True)

https://mp.weixin.qq.com/s?__biz=MzI0NDcxNzc5Mg==&mid=2247483844&idx=1&sn=d7172490cbe80ea03c63ba0ba9417235&chksm=e958c04bde2f495dd777573eb11541a958d4adffc8a7a63cd0540e38a44bbb84bf181a992010#rd
Title:  [inc]How to Become the Best in the World at What You Do
write in to file: http://y.biomooc.com/wangjl/en20171117.html
Not exist the date
rollback
(<class 'pymysql.err.DataError'>, DataError(1406, "Data too long for column 'msg_content' at row 1"), <traceback object at 0x000002E96437BC48>)
580 3.76869175193693


## reRun 补充